In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt

/home/ruben/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (12,12)
mineProbability = 0.2      # Probability that a square contain a mine
missingProbability = 0.1   # Probability that a square is missing adjacency info

In [3]:
# count the number of mines in the proximity of given square, including square itself
def countMines(board,r,c):
    count = 0
    rows, cols = board.shape
    for i in [r-1,r,r+1]:
        if i >= 0 and i < rows:
            for j in [c-1,c,c+1]:
                if j >= 0 and j < cols:
                    count += int(board[i,j])
    return count

In [4]:
def minesweepMatrix(dimensions):
    rows,cols = dimensions
    size = rows * cols
    A = np.zeros([size,size],dtype=int)
    for rA in range(size):
        for cA in range(size):
            inRow, inCol = divmod(rA,cols)
            outRow, outCol = divmod(cA,cols)
            A[rA,cA] = abs(inRow-outRow) <= 1 and abs(inCol-outCol) <= 1
    return(A)

In [5]:
# Converts a board of mines into a board of mine counts
'''
def boardMineCounts_(board):
    mineInfo = np.zeros(board.shape, dtype = int)
    rows, cols = board.shape
    for i in range(rows):
        for j in range(cols):
            mineInfo[i,j] = countMines(board,i,j)
    return mineInfo
'''
def boardMineCounts(board):
    return(minesweepMatrix(board.shape).dot(board.flatten()).reshape(board.shape))

In [6]:
def boardPartialMineCounts(board):
    result = boardMineCounts(board)
    for index, x in np.ndenumerate(board):
        if x: result[index] = -1
        elif r.uniform(0, 1) < missingProbability: result[index] = -1
    return result

In [7]:
# Generates a random training batch of size at most n
def next_training_batch(n):
    batch_xs = []
    batch_ys = []
    for _ in range(n):
        board = np.random.random(dimensions) < mineProbability
        counts = boardPartialMineCounts(board)
        frees = board.flatten().astype(float)
        freesSum = sum(frees)
        if freesSum > 0:
            batch_xs.append(counts.flatten())
            batch_ys.append(frees / freesSum)
    return (np.asarray(batch_xs), np.asarray(batch_ys))

In [8]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCounts = tf.placeholder(tf.int32, [None, size], name="mineCounts")
mineCountsOneHot = tf.reshape(tf.one_hot(mineCounts+1,10), [-1, size*10])
W = tf.Variable(tf.random_normal([size*10, size], stddev=0.01), name="W")
b = tf.Variable(tf.random_normal([size], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [9]:
mineFreeAverages = tf.placeholder(tf.float32, [None, size], name="mineFreeAverages")

In [10]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=mineFreeAverages, logits=y))

In [11]:
# Summaries for tensorboard
with tf.name_scope('W_reshape'):
    image_shaped_W = tf.reshape(W, [-1, size*10, size, 1])
    tf.summary.image('W', image_shaped_W, 1000)

with tf.name_scope('b_reshape'):
    image_shaped_b = tf.reshape(b, [-1, rows, cols, 1])
    tf.summary.image('b', image_shaped_b, 1000)

_ = tf.summary.scalar('accuracy', cross_entropy)

In [12]:
# Optimiser
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [13]:
# Create session and initialise or restore stuff
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('.', sess.graph)

In [14]:
tf.global_variables_initializer().run()

In [15]:
# Restore model?
#saver.restore(sess, "./saves.tf.Mines3/model-1000")

In [16]:
# Train
for iteration in range(10001):
    batch_xs, batch_ys = next_training_batch(100)
    if iteration % 10 == 0:
        summary, acc, _ = sess.run([merged, cross_entropy, train_step],
                                   feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
        writer.add_summary(summary, iteration)
        print('%s: Accuracy at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, acc))
    else:
        _ = sess.run(train_step, feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
    if iteration % 1000 == 0:
        save_path = saver.save(sess, './saves.tf.Mines3/model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-05 14:55:20: Accuracy at step 0: 4.97697
Model saved in file: ./saves.tf.Mines3/model-0
2017-11-05 14:55:26: Accuracy at step 10: 4.94447
2017-11-05 14:55:33: Accuracy at step 20: 4.90134
2017-11-05 14:55:39: Accuracy at step 30: 4.87258
2017-11-05 14:55:46: Accuracy at step 40: 4.83213
2017-11-05 14:55:53: Accuracy at step 50: 4.79532
2017-11-05 14:55:59: Accuracy at step 60: 4.76595
2017-11-05 14:56:06: Accuracy at step 70: 4.73029
2017-11-05 14:56:12: Accuracy at step 80: 4.69795
2017-11-05 14:56:19: Accuracy at step 90: 4.66942
2017-11-05 14:56:25: Accuracy at step 100: 4.636
2017-11-05 14:56:32: Accuracy at step 110: 4.62983
2017-11-05 14:56:39: Accuracy at step 120: 4.58871
2017-11-05 14:56:45: Accuracy at step 130: 4.56895
2017-11-05 14:56:52: Accuracy at step 140: 4.53597
2017-11-05 14:56:58: Accuracy at step 150: 4.51945
2017-11-05 14:57:05: Accuracy at step 160: 4.48905
2017-11-05 14:57:12: Accuracy at step 170: 4.46237
2017-11-05 14:57:19: Accuracy at step 180: 4.453

2017-11-05 15:12:46: Accuracy at step 1590: 3.63938
2017-11-05 15:12:52: Accuracy at step 1600: 3.63946
2017-11-05 15:12:59: Accuracy at step 1610: 3.625
2017-11-05 15:13:06: Accuracy at step 1620: 3.61432
2017-11-05 15:13:12: Accuracy at step 1630: 3.62799
2017-11-05 15:13:19: Accuracy at step 1640: 3.62861
2017-11-05 15:13:25: Accuracy at step 1650: 3.60487
2017-11-05 15:13:32: Accuracy at step 1660: 3.60726
2017-11-05 15:13:38: Accuracy at step 1670: 3.5966
2017-11-05 15:13:45: Accuracy at step 1680: 3.63683
2017-11-05 15:13:51: Accuracy at step 1690: 3.62575
2017-11-05 15:13:58: Accuracy at step 1700: 3.63093
2017-11-05 15:14:05: Accuracy at step 1710: 3.60064
2017-11-05 15:14:11: Accuracy at step 1720: 3.60949
2017-11-05 15:14:18: Accuracy at step 1730: 3.5992
2017-11-05 15:14:24: Accuracy at step 1740: 3.59065
2017-11-05 15:14:31: Accuracy at step 1750: 3.62008
2017-11-05 15:14:37: Accuracy at step 1760: 3.57839
2017-11-05 15:14:44: Accuracy at step 1770: 3.6237
2017-11-05 15:14:

2017-11-05 15:29:54: Accuracy at step 3160: 3.55334
2017-11-05 15:30:01: Accuracy at step 3170: 3.53979
2017-11-05 15:30:08: Accuracy at step 3180: 3.5191
2017-11-05 15:30:14: Accuracy at step 3190: 3.54313
2017-11-05 15:30:21: Accuracy at step 3200: 3.58383
2017-11-05 15:30:27: Accuracy at step 3210: 3.54164
2017-11-05 15:30:34: Accuracy at step 3220: 3.48779
2017-11-05 15:30:40: Accuracy at step 3230: 3.53562
2017-11-05 15:30:47: Accuracy at step 3240: 3.53
2017-11-05 15:30:54: Accuracy at step 3250: 3.51465
2017-11-05 15:31:00: Accuracy at step 3260: 3.53539
2017-11-05 15:31:07: Accuracy at step 3270: 3.52639
2017-11-05 15:31:13: Accuracy at step 3280: 3.53073
2017-11-05 15:31:20: Accuracy at step 3290: 3.54531
2017-11-05 15:31:26: Accuracy at step 3300: 3.56881
2017-11-05 15:31:33: Accuracy at step 3310: 3.51467
2017-11-05 15:31:39: Accuracy at step 3320: 3.55237
2017-11-05 15:31:46: Accuracy at step 3330: 3.53398
2017-11-05 15:31:52: Accuracy at step 3340: 3.4916
2017-11-05 15:31:

2017-11-05 15:47:10: Accuracy at step 4740: 3.528
2017-11-05 15:47:17: Accuracy at step 4750: 3.50305
2017-11-05 15:47:23: Accuracy at step 4760: 3.51604
2017-11-05 15:47:30: Accuracy at step 4770: 3.47011
2017-11-05 15:47:36: Accuracy at step 4780: 3.4812
2017-11-05 15:47:43: Accuracy at step 4790: 3.49812
2017-11-05 15:47:49: Accuracy at step 4800: 3.5182
2017-11-05 15:47:56: Accuracy at step 4810: 3.51172
2017-11-05 15:48:02: Accuracy at step 4820: 3.51104
2017-11-05 15:48:09: Accuracy at step 4830: 3.52036
2017-11-05 15:48:15: Accuracy at step 4840: 3.47574
2017-11-05 15:48:22: Accuracy at step 4850: 3.48888
2017-11-05 15:48:29: Accuracy at step 4860: 3.49012
2017-11-05 15:48:35: Accuracy at step 4870: 3.52077
2017-11-05 15:48:42: Accuracy at step 4880: 3.53204
2017-11-05 15:48:48: Accuracy at step 4890: 3.47747
2017-11-05 15:48:55: Accuracy at step 4900: 3.49741
2017-11-05 15:49:01: Accuracy at step 4910: 3.46996
2017-11-05 15:49:08: Accuracy at step 4920: 3.51774
2017-11-05 15:49

2017-11-05 16:04:12: Accuracy at step 6300: 3.47041
2017-11-05 16:04:19: Accuracy at step 6310: 3.47088
2017-11-05 16:04:25: Accuracy at step 6320: 3.52391
2017-11-05 16:04:32: Accuracy at step 6330: 3.50952
2017-11-05 16:04:38: Accuracy at step 6340: 3.48758
2017-11-05 16:04:45: Accuracy at step 6350: 3.4468
2017-11-05 16:04:51: Accuracy at step 6360: 3.45571
2017-11-05 16:04:58: Accuracy at step 6370: 3.51753
2017-11-05 16:05:05: Accuracy at step 6380: 3.4967
2017-11-05 16:05:11: Accuracy at step 6390: 3.51818
2017-11-05 16:05:18: Accuracy at step 6400: 3.49281
2017-11-05 16:05:24: Accuracy at step 6410: 3.47334
2017-11-05 16:05:31: Accuracy at step 6420: 3.45686
2017-11-05 16:05:37: Accuracy at step 6430: 3.50136
2017-11-05 16:05:44: Accuracy at step 6440: 3.47884
2017-11-05 16:05:50: Accuracy at step 6450: 3.47195
2017-11-05 16:05:57: Accuracy at step 6460: 3.4594
2017-11-05 16:06:03: Accuracy at step 6470: 3.49001
2017-11-05 16:06:10: Accuracy at step 6480: 3.51195
2017-11-05 16:0

2017-11-05 16:21:20: Accuracy at step 7870: 3.50612
2017-11-05 16:21:27: Accuracy at step 7880: 3.47915
2017-11-05 16:21:33: Accuracy at step 7890: 3.48634
2017-11-05 16:21:40: Accuracy at step 7900: 3.48925
2017-11-05 16:21:46: Accuracy at step 7910: 3.49414
2017-11-05 16:21:53: Accuracy at step 7920: 3.4785
2017-11-05 16:22:00: Accuracy at step 7930: 3.4812
2017-11-05 16:22:06: Accuracy at step 7940: 3.45648
2017-11-05 16:22:13: Accuracy at step 7950: 3.46408
2017-11-05 16:22:19: Accuracy at step 7960: 3.49334
2017-11-05 16:22:26: Accuracy at step 7970: 3.51273
2017-11-05 16:22:32: Accuracy at step 7980: 3.49727
2017-11-05 16:22:39: Accuracy at step 7990: 3.50018
2017-11-05 16:22:45: Accuracy at step 8000: 3.45578
Model saved in file: ./saves.tf.Mines3/model-8000
2017-11-05 16:22:52: Accuracy at step 8010: 3.50683
2017-11-05 16:22:59: Accuracy at step 8020: 3.46779
2017-11-05 16:23:05: Accuracy at step 8030: 3.48246
2017-11-05 16:23:12: Accuracy at step 8040: 3.4793
2017-11-05 16:23:

2017-11-05 16:38:33: Accuracy at step 9440: 3.46401
2017-11-05 16:38:39: Accuracy at step 9450: 3.47601
2017-11-05 16:38:46: Accuracy at step 9460: 3.44789
2017-11-05 16:38:53: Accuracy at step 9470: 3.48403
2017-11-05 16:39:00: Accuracy at step 9480: 3.46668
2017-11-05 16:39:06: Accuracy at step 9490: 3.4742
2017-11-05 16:39:13: Accuracy at step 9500: 3.47906
2017-11-05 16:39:20: Accuracy at step 9510: 3.50061
2017-11-05 16:39:26: Accuracy at step 9520: 3.44061
2017-11-05 16:39:33: Accuracy at step 9530: 3.47333
2017-11-05 16:39:39: Accuracy at step 9540: 3.46145
2017-11-05 16:39:46: Accuracy at step 9550: 3.44988
2017-11-05 16:39:52: Accuracy at step 9560: 3.46586
2017-11-05 16:39:59: Accuracy at step 9570: 3.48708
2017-11-05 16:40:06: Accuracy at step 9580: 3.48058
2017-11-05 16:40:12: Accuracy at step 9590: 3.45537
2017-11-05 16:40:19: Accuracy at step 9600: 3.46122
2017-11-05 16:40:25: Accuracy at step 9610: 3.47454
2017-11-05 16:40:32: Accuracy at step 9620: 3.47262
2017-11-05 16

In [17]:
# Test trained model
batch_xs, batch_ys = next_training_batch(1000)
print(sess.run(cross_entropy, feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys}))

3.47817


In [40]:
# Run a test
batchSize = 100000
batch_xs, batch_ys = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
board = (batch_ys == 0).astype(int)
frees = [board[i][bestSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(frees))

Number of errors for batch size of  10000
4
